In [1]:
import json
from StringIO import StringIO
import pandas as pd
import openpyxl

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')

In [6]:
active_businesses = pd.read_csv('Datasources/sdcounty_datasets/sd_active_businesses_datasd.csv')

In [7]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [8]:
schema_two = StructType([StructField("business_id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [9]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [10]:
spql_active_businesses_info = sqlContext.createDataFrame(active_businesses[['account_key', 'lon', 'lat']], schema_two)

In [11]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [12]:
spql_active_businesses_info_rdd = spql_active_businesses_info.rdd.map(lambda row:(row.business_id, row.longitude,row.latitude))

In [13]:
spql_listings_and_businesses_info_rdd = spql_listings_info_rdd.cartesian(spql_active_businesses_info_rdd)

In [14]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [15]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [16]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_rdd.map(distance_calculation)

In [17]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField('business_id', IntegerType(), True),
                    StructField("business_longitude", DoubleType(), True),
                    StructField("business_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [18]:
spql_listings_and_businesses_info_df = sqlContext.createDataFrame(spql_listings_and_businesses_info_rdd, schema_three)

In [19]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.business_id, row.business_longitude, row.business_latitude, row.distance))

In [20]:
closest_business_distance = spql_listings_and_businesses_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [21]:
#closest_business_distance = spql_listings_and_businesses_info_rdd.cartesian(closest_business_distance).filter(lambda x: x[0][-1] == x[1][-1]).map(lambda x: (x[0][0], x[0][-1], x[0][3]))

In [22]:
closest_businesses_count_one = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
closest_businesses_count_three = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
closest_businesses_count_five = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
closest_businesses_count_ten = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [23]:
closest_businesses_count_sixteen = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [24]:
count = 0

In [25]:
import datetime
today = datetime.date.today()
count+=1
filename = 'Datasources/listings_businesses/listings_businesses_' + str(today) + '_V' + str(count) + '.csv'

In [26]:
print filename

Datasources/listings_businesses/listings_businesses_2018-05-27_V1.csv


In [27]:
df1 = closest_businesses_count_one.toDF()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at scala.collection.

In [ ]:
df2 = closest_businesses_count_three.toDF()

In [ ]:
df3 = closest_businesses_count_five.toDF()

In [ ]:
df4 = closest_business_distance.toDF()

In [ ]:
df1 = df1.toPandas()

In [ ]:
df1 = df1.rename(columns={'_1':'listing_id', '_2':'business_count_1km'})

In [ ]:
df2 = df2.toPandas()

In [ ]:
df2 = df2.rename(columns={'_1':'listing_id', '_2':'business_count_3km'})

In [ ]:
df3 = df3.toPandas()

In [ ]:
df3 = df3.rename(columns={'_1':'listing_id', '_2':'business_count_5km'})

In [ ]:
df4 = df4.toPandas()

In [ ]:
df4 = df4.rename(columns={'_1':'listing_id', '_2':'distance_to_closest_business_km', '_3': 'account_key_of_closest_distance'})

In [ ]:
t1 = df1.merge(df2, how='inner', on='listing_id')

In [ ]:
t2 = t1.merge(df3, how='inner', on='listing_id')
t3 = t2.merge(df4, how='inner', on='listing_id')

In [ ]:
t3.head()

In [ ]:
t3 = t3.merge(active_businesses[['account_key', 'doing_bus_as_name', 'ownership_type']], how='inner', left_on='account_key_of_closest_distance', right_on = 'account_key')

In [ ]:
t3 = t3.dropna()

In [ ]:
t3 = t3.drop('account_key', axis = 1)

In [ ]:
t3.to_csv(filename,index=False)

In [ ]:
t3.head()

In [28]:
sc.stop()